# Linear Regression

In [4]:
import pandas as pd
import numpy as np

import pickle

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.api import OLS, add_constant
from statsmodels.tools.eval_measures import aic, bic

from sklearn.linear_model import LinearRegression

In [6]:
# load the data abouses houses divided into train and test samples
# prepared and saved earlier - BEFORE encoding of categorical predictors

with open('data/houses_prepared.pkl', 'rb') as f:
    houses_train = pickle.load(f)
    houses_test = pickle.load(f)

houses_test.head()

,Order,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Alley,Lot_Shape,Land_Contour,Lot_Config,...,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Fence,Misc_Feature,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price
0,1,526301100,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,No_Alley_Access,Slightly_Irregular,Lvl,Corner,...,Partial_Pavement,210,62,No_Fence,None,5,2010,WD,Normal,215000
1,2,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,No_Alley_Access,Regular,Lvl,Inside,...,Paved,140,0,Minimum_Privacy,None,6,2010,WD,Normal,105000
2,3,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,No_Alley_Access,Slightly_Irregular,Lvl,Corner,...,Paved,393,36,No_Fence,Other,6,2010,WD,Normal,172000
3,4,526353030,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,No_Alley_Access,Regular,Lvl,Corner,...,Paved,0,0,No_Fence,None,4,2010,WD,Normal,244000
4,5,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,No_Alley_Access,Slightly_Irregular,Lvl,Inside,...,Paved,212,34,Minimum_Privacy,None,3,2010,WD,Normal,189900


## Model estimation

Let's estimate the first model. Since we will be interested in the interpretation of individual parameters, not just prediction, we will use the methods from the `statsmodels` library. In terms of code syntax, estimating a model requires providing its formula. The model formula is given in the `dependent_variable ~ independent_variables` system, e.g. `Y ~ X` for simple regression with one explained variable ($X$).

If we want to include more explanatory variables, they should be separated in the model formula with pluses "`+`", e.g. `Y ~ X1 + X2 + X3`.

Now let's estimate a model in which the only explanatory variable is total area above the ground (simple regression).

Let's save the modeling result as an object:

In [9]:
# Total area above the ground as the only predictor
houses_model1 = smf.ols(formula = 'Sale_Price ~ Gr_Liv_Area',
                        data = houses_train).fit() 
# fit method estimates the coefficients

Alternatively, without specifying a formula, which is useful in the case of multiple variables X's, including decoded qualitative ones:

In [12]:
X = sm.add_constant(houses_train['Gr_Liv_Area']) # adding a constant to X matrix
houses_model1 = sm.OLS(houses_train['Sale_Price'], X).fit()

# however, OLS function requires all variables to be numeric, so we need to convert
# categorical variables into dummies

# ols function DOES allow for categorical variables, but they need to be specified
# as such in the fomula  (with C() )

In [14]:
print(houses_model1.summary())

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     2039.
Date:                Fri, 17 Oct 2025   Prob (F-statistic):          7.22e-305
Time:                        16:10:38   Log-Likelihood:                -23980.
No. Observations:                1941   AIC:                         4.796e+04
Df Residuals:                    1939   BIC:                         4.797e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         1.68e+04   3875.120      4.334      

---
In the lower part of results we can see estimates of individual model parameters (only two here), their standard errors, t statistics, their p-values and 95% confidence intervals

It can be seen that `Gr_Liv_Area` DOES significantly affect the price of a house having p-value (`P>|t|`) = 0.000

On top of the results we also have the displayed value:

* `R-squared`: Multiple R-squared: 0.513
* `Adj. R-squared`: Adjusted R-squared: 0.512

which will be described in details on the next labs

* `F-statistic`: testing the joint significance of the model and its p-value: F-statistic: 2039,  p-value: < 7.22e-305
* `AIC`: Akaike Information Criterion
* `BIC`: Bayesian Information Criterion

In [21]:
# estimate the model with mode explanatory variables (quantitative)
houses_model2 = smf.ols(
    formula = 'Sale_Price ~ Gr_Liv_Area + Garage_Area + Full_Bath + Year_Built + TotRms_AbvGrd',
    data = houses_train
).fit()

print(houses_model2.summary())

# R-squared -> variability of the Y
# using more variables we explain 69% of the variability of the Sale_Price

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     876.7
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        16:23:14   Log-Likelihood:                -23528.
No. Observations:                1941   AIC:                         4.707e+04
Df Residuals:                    1935   BIC:                         4.710e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.574e+06   8.05e+04    -19.559

---
* Using more variables we explain 69% of the variability of the `Sale_Price` (R2)!

* all used variables are statistically significant except for `Full_Bath`

* `AIC` and `BIC` are lower than for model 1

In [25]:
# add some qualitative explanatory variables to the model (categorical Xi)
# the ols() function does not require the conversion of categorical Xi into dummies
# it is done automatically using C() function.

houses_model3 = smf.ols(
    formula = 'Sale_Price ~ Gr_Liv_Area + Garage_Area + Full_Bath + Year_Built + TotRms_AbvGrd + C(Exter_Qual) + C(Kitchen_Qual) + C(Overall_Qual) + C(Central_Air) + C(Foundation)',
    data = houses_train
).fit()

print(houses_model3.summary())
# now R-squared shows 81% meaning adding more categorical variables explains Y more

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     342.2
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        16:28:44   Log-Likelihood:                -23028.
No. Observations:                1941   AIC:                         4.611e+04
Df Residuals:                    1915   BIC:                         4.625e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

---
* `R-squared` has further increased to 0.817

* `AIC` and `BIC` decreased as compared to model2

---

In [30]:
# model with all variables
# ! create a formula automatically ensuring that all categorical variables -
# are treated properly 

# it is only possible within the ols() function

houses_train2 = houses_train.drop(columns = ['Order', 'PID'])

houses_model4 = smf.ols(
    formula = 'Sale_Price ~ ' + ' + '.join(f'C({col})' if houses_train2[col].dtype == 'object' else col for col in houses_train2.columns if col != 'Sale_Price'),
    data = houses_train2
).fit()

print(houses_model4.summary())

# we achived 90% of R-squared

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     74.14
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        16:39:11   Log-Likelihood:                -22428.
No. Observations:                1941   AIC:                         4.528e+04
Df Residuals:                    1727   BIC:                         4.648e+04
Df Model:                         213                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [34]:
# load the data about houses divided into train and test samples
# prepared and saved earlier - AFTER encoding of categorical predictors

with open('data/houses_prepared_encoded.pkl', 'rb') as f:
    houses_train_encoded = pickle.load(f)
    houses_test_encoded = pickle.load(f)

houses_test_encoded.head()

,Order,PID,Lot_Frontage,Lot_Area,Lot_Shape,Land_Slope,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,...,Sale_Type_New,Sale_Type_Other,Sale_Type_WD,Sale_Condition_Family,Sale_Condition_Normal,Sale_Condition_Other,Sale_Condition_Partial,Land_Contour_HLS,Land_Contour_Low,Land_Contour_Lvl
0,1,526301100,141,31770,1.0,0.0,4.0,5.0,1960,1960,...,0,0,1,0,1,0,0,0,0,1
1,2,526350040,80,11622,0.0,0.0,5.0,4.0,1961,1961,...,0,0,1,0,1,0,0,0,0,1
2,3,526351010,81,14267,1.0,0.0,4.0,4.0,1958,1958,...,0,0,1,0,1,0,0,0,0,1
3,4,526353030,93,11160,0.0,0.0,3.0,5.0,1968,1968,...,0,0,1,0,1,0,0,0,0,1
4,5,527105010,74,13830,1.0,0.0,5.0,5.0,1997,1998,...,0,0,1,0,1,0,0,0,0,1


In [40]:
# although ordinal features were recoded with the OrdinalEncoder,
# they still have the type "object" in the DataFrame

houses_train_encoded.select_dtypes(include = 'object').head()

,Lot_Shape,Land_Slope,Overall_Qual,Overall_Cond,Exter_Qual,Exter_Cond,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_Type_2,Heating_QC,Electrical,Kitchen_Qual,Fireplace_Qu,Garage_Finish,Garage_Qual,Garage_Cond,Paved_Drive,Fence
989,1.0,0.0,4.0,2.0,0.0,2.0,1.0,2.0,3.0,1.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,4.0
990,1.0,0.0,4.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,0.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,4.0
991,0.0,0.0,4.0,5.0,2.0,2.0,3.0,2.0,2.0,1.0,4.0,2.0,0.0,2.0,1.0,2.0,2.0,2.0,0.0,4.0
992,1.0,0.0,4.0,5.0,2.0,2.0,2.0,2.0,3.0,2.0,5.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,4.0
993,0.0,0.0,4.0,5.0,2.0,2.0,1.0,2.0,3.0,1.0,5.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0,4.0


In [42]:
houses_train_encoded.select_dtypes(include = 'object').columns

Index(['Lot_Shape', 'Land_Slope', 'Overall_Qual', 'Overall_Cond', 'Exter_Qual',
       'Exter_Cond', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating_QC', 'Electrical',
       'Kitchen_Qual', 'Fireplace_Qu', 'Garage_Finish', 'Garage_Qual',
       'Garage_Cond', 'Paved_Drive', 'Fence'],
      dtype='object')

In [48]:
# to make sure that the type of the variable is correct, we can convert it to 
# the numeric type

# ensures all columns are numeric
houses_train_encoded = houses_train_encoded.apply(pd.to_numeric) 

houses_train_encoded.select_dtypes(include = 'object').columns

Index([], dtype='object')

In [52]:
# now we can prepare the data for the model
X_train = sm.add_constant(
    houses_train_encoded.drop(columns = ['Sale_Price', 'Order', 'PID'])
)

y_train = houses_train_encoded['Sale_Price']

# we will use these data for a simplified backward elimination procedure
# using the AIC or BIC criterion to select the best model

In [54]:
model5_full = sm.OLS(y_train, X_train).fit()

print(model5_full.summary())

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     90.99
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        16:54:24   Log-Likelihood:                -22598.
No. Observations:                1941   AIC:                         4.549e+04
Df Residuals:                    1792   BIC:                         4.632e+04
Df Model:                         148                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [58]:
# define a function that will perform backward elimination
# using AIC or BIS as the criterion

def backward_elimination_aic_bic(X, y, criterion = 'AIC'):
    '''
    Perform backward elimination using AIC or BIC as the criterion.

    Parameters:
        X (DataFrame): Feature matrix with a constant column.
        y (Series): Target variable.
        criterion (str): 'AIC' or 'BIC' (default: 'AIC').

    Returns:
        statsmodels OLS fitted model with selected features.
    '''
    model = sm.OLS(y, X).fit()

    while len(X.columns) > 1: # at least one predictor + constant
        best_criterion = model.aic if criterion == 'AIC' else model.bic

        # compute AIC/BIC for models without each predictor
        aic_bic_values = {}
        for col in X.columns[1:]:  # skip intercept
            X_new = X.drop(columns = [col])
            new_model = sm.OLS(y, X_new).fit()
            aic_bic_values[col] = new_model.aic if criterion == 'AIC' else new_model.bic

        # find the feature whose removal lowers AIC/BIC the most
        worst_feature = min(aic_bic_values, key = aic_bic_values.get)
        worst_aic_bic = aic_bic_values[worst_feature]

        # stop if no improvement
        if worst_aic_bic >= best_criterion:
            break

        # remove the feature and update the model
        X = X.drop(columns = [worst_feature])
        model = sm.OLS(y, X).fit()

    return model

In [ ]:
# !!! Caution !!! This function may take 5 mins to run

# perform backward elimination using AIC
houses_model5_AIC = backward_elimination_aic_bic(X_train, y_train, criterion = 'AIC')

# save the model as a pickle file
with open('data/houses_model5_AIC.pkl', 'wb') as f:
    pickle.dump(houses_model5_AIC, f)

In [60]:
# load the model from the file
with open('data/houses_model5_AIC.pkl', 'rb') as f:
    houses_model5_AIC = pickle.load(f)

print(houses_model5_AIC.summary())

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     177.7
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        17:15:32   Log-Likelihood:                -22618.
No. Observations:                1941   AIC:                         4.539e+04
Df Residuals:                    1863   BIC:                         4.583e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [ ]:
# !!! CAUTION !!! This function may take ca 5 mins to run

# Perform backward elimination using BIC
houses_model5_BIC = backward_elimination_aic_bic(X_train, y_train,
                                                 criterion = 'BIC')

# save the model as a pickle file
with open('data/houses_model5_BIC.pkl', 'wb') as f:
    pickle.dump(houses_model5_BIC, f)

In [62]:
# load the model from the file
with open('data/houses_model5_BIC.pkl', 'rb') as f:
    houses_model5_BIC = pickle.load(f)

print(houses_model5_BIC.summary())

                            OLS Regression Results                            
Dep. Variable:             Sale_Price   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     271.7
Date:                Fri, 17 Oct 2025   Prob (F-statistic):               0.00
Time:                        17:18:49   Log-Likelihood:                -22672.
No. Observations:                1941   AIC:                         4.544e+04
Df Residuals:                    1892   BIC:                         4.571e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [66]:
# Do both approaches give the same coefficients?

houses_model5_AIC.params.equals(houses_model5_BIC.params)

# No !!!

False

In [68]:
# which variables retained based on AIC were not selected by BIC?

set(houses_model5_AIC.params.index) - set(houses_model5_BIC.params.index)

{'Condition_1_Other',
 'Condition_1_PosN',
 'Condition_1_RRAn',
 'Exterior_1st_Other',
 'Exterior_2nd_BrkFace',
 'Exterior_2nd_MetalSd',
 'Exterior_2nd_Wd Shng',
 'First_Flr_SF',
 'Foundation_PConc',
 'Full_Bath',
 'Garage_Area',
 'Garage_Type_Other',
 'Half_Bath',
 'Heating_QC',
 'House_Style_One_and_Half_Fin',
 'House_Style_Other',
 'House_Style_SFoyer',
 'House_Style_SLvl',
 'House_Style_Two_Story',
 'Lot_Area',
 'Lot_Config_FR2',
 'Mas_Vnr_Type_Stone',
 'Neighborhood_Meadow_Village',
 'Neighborhood_Old_Town',
 'Open_Porch_SF',
 'Sale_Condition_Normal',
 'Sale_Condition_Other',
 'TotRms_AbvGrd',
 'Year_Sold'}

In [70]:
# were there any variables retained based on BIC that were not selected by AIC?

set(houses_model5_BIC.params.index) - set(houses_model5_AIC.params.index)

# No! As expected, BIC was more restrictive than AIC

set()